# PART 5. 데이터 사전 처리

# 1. 누락 데이터 처리

(1) 누락 데이터 확인

In [1]:
import pandas as pd
import seaborn as sns

df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [3]:
nan_deck=df['deck'].value_counts(dropna=False)
print(nan_deck)

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64


In [4]:
print(df.head().isnull())  #deck열에 누락데이터 존재

   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  


In [5]:
print(df.head().notnull())

   survived  pclass   sex   age  sibsp  parch  fare  embarked  class   who  \
0      True    True  True  True   True   True  True      True   True  True   
1      True    True  True  True   True   True  True      True   True  True   
2      True    True  True  True   True   True  True      True   True  True   
3      True    True  True  True   True   True  True      True   True  True   
4      True    True  True  True   True   True  True      True   True  True   

   adult_male   deck  embark_town  alive  alone  
0        True  False         True   True   True  
1        True   True         True   True   True  
2        True  False         True   True   True  
3        True   True         True   True   True  
4        True  False         True   True   True  


In [6]:
print(df.head().isnull().sum(axis=0))  #누락 데이터인 경우 1반환,합한 것

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           3
embark_town    0
alive          0
alone          0
dtype: int64


(2) 누락 데이터 제거

In [7]:
missing_df=df.isnull()
for col in missing_df.columns:
    missing_count=missing_df[col].value_counts()   #각 열의 nan 개수 파악
    
    try:
        print(col,':', missing_count[True])
    except:
        print(col, ':', 0)

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


In [8]:
df_thresh=df.dropna(axis=1, thresh=500)
print(df_thresh.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')


In [9]:
df_age=df.dropna(subset=['age'],how='any',axis=0)
print(len(df_age))

714


(3) 누락 데이터 치환

In [10]:
df=sns.load_dataset('titanic')
mean_age=df['age'].mean(axis=0)
df['age'].fillna(mean_age, inplace=True)

print(df['age'].head(10))

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: age, dtype: float64


In [11]:
df=sns.load_dataset('titanic')
most_freq=df['embark_town'].value_counts(dropna=True).idxmax()
print(most_freq)

df['embark_town'].fillna(most_freq,inplace=True)

Southampton


In [12]:
df=sns.load_dataset('titanic')
df['embark_town'].fillna(method='ffill',inplace=True)
print(df['embark_town'][825:830])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
Name: embark_town, dtype: object


# 2. 중복 데이터 처리

(1) 중복 데이터 확인

In [14]:
df=pd.DataFrame({'c1':['a','a','b','a','b'],
                'c2':[1,1,1,2,2],
                'c3':[1,1,2,2,2]})
print(df)

df_dup=df.duplicated()
print(df_dup)

col_dup=df['c2'].duplicated()
print(col_dup)

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2
0    False
1     True
2    False
3    False
4    False
dtype: bool
0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool


(2) 중복 데이터 제거

In [15]:
df2=df.drop_duplicates()
print(df2)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2
4  b   2   2


In [16]:
#c2, c3 열을 기준으로 제거
df3=df.drop_duplicates(subset=['c2','c3'])
print(df3)

  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   2   2


# 3. 데이터 표준화

(1) 단위환산

In [21]:
df=pd.read_csv('C:/Users/user/Desktop/쿠글 여름 스터디/2주차 자료/part4/auto-mpg.csv',header=None)
df.columns=['mpg','cylinders','displacement','horsepower','weigh','acceleration','model year','origin','name']
print(df.head(3))

#mpg를 kpl로 변환(단위변환)
mpg_to_kpl=1.60934/3.78541
df['kpl']=df['mpg']*mpg_to_kpl
print(df.head(3))
df['kpl']=df['kpl'].round(2)
print(df.head(3))

    mpg  cylinders  displacement horsepower   weigh  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
    mpg  cylinders  displacement horsepower   weigh  acceleration  model year  \
0  18.0          8         307.0      130.0  3504.0          12.0          70   
1  15.0          8         350.0      165.0  3693.0          11.5          70   
2  18.0          8         318.0      150.0  3436.0          11.0          70   

   origin                       name       kpl  
0       1  chevrolet chevelle malibu  7.652571  
1       1          buick skylark 320  6.377143  
2       1         plymouth satellite  7.652571 

(2) 자료형 변환

In [37]:
df=pd.read_csv('C:/Users/user/Desktop/쿠글 여름 스터디/2주차 자료/part4/auto-mpg.csv',header=None)
df.columns=['mpg','cylinders','displacement','horsepower','weigh','acceleration','model year','origin','name']

print(df.dtypes)
print('\n')

mpg             float64
cylinders         int64
displacement    float64
horsepower       object
weigh           float64
acceleration    float64
model year        int64
origin            int64
name             object
dtype: object




In [38]:
print(df['horsepower'].unique())

['130.0' '165.0' '150.0' '140.0' '198.0' '220.0' '215.0' '225.0' '190.0'
 '170.0' '160.0' '95.00' '97.00' '85.00' '88.00' '46.00' '87.00' '90.00'
 '113.0' '200.0' '210.0' '193.0' '?' '100.0' '105.0' '175.0' '153.0'
 '180.0' '110.0' '72.00' '86.00' '70.00' '76.00' '65.00' '69.00' '60.00'
 '80.00' '54.00' '208.0' '155.0' '112.0' '92.00' '145.0' '137.0' '158.0'
 '167.0' '94.00' '107.0' '230.0' '49.00' '75.00' '91.00' '122.0' '67.00'
 '83.00' '78.00' '52.00' '61.00' '93.00' '148.0' '129.0' '96.00' '71.00'
 '98.00' '115.0' '53.00' '81.00' '79.00' '120.0' '152.0' '102.0' '108.0'
 '68.00' '58.00' '149.0' '89.00' '63.00' '48.00' '66.00' '139.0' '103.0'
 '125.0' '133.0' '138.0' '135.0' '142.0' '77.00' '62.00' '132.0' '84.00'
 '64.00' '74.00' '116.0' '82.00']


In [39]:
import numpy as np
df['horsepower'].replace('?',np.nan, inplace=True)  #'?'를 nan 값으로 변환
df.dropna(subset=['horsepower'],axis=0, inplace=True)   # nan값 제거
df['horsepower']=df['horsepower'].astype('float')

print(df['horsepower'].dtypes)

float64


In [40]:
print(df['origin'].unique())

df['origin'].replace({1:'USA',2:'EU',3:'JPN'},inplace=True)

print(df['origin'].unique())
print(df['origin'].dtypes)

[1 3 2]
['USA' 'JPN' 'EU']
object


In [41]:
df['origin']=df['origin'].astype('category')
print(df['origin'].dtypes)

df['origin']=df['origin'].astype('str')
print(df['origin'].dtypes)

category
object


In [42]:
#연도를 범주형으로 변환
print(df['model year'].head(10))
df['model year']=df['model year'].astype('category')
print(df['model year'].head(10))

0    70
1    70
2    70
3    70
4    70
5    70
6    70
7    70
8    70
9    70
Name: model year, dtype: int64
0    70
1    70
2    70
3    70
4    70
5    70
6    70
7    70
8    70
9    70
Name: model year, dtype: category
Categories (13, int64): [70, 71, 72, 73, ..., 79, 80, 81, 82]


# 4. 범주형 데이터 처리

(1) 구간 분할

In [43]:
df=pd.read_csv('C:/Users/user/Desktop/쿠글 여름 스터디/2주차 자료/part4/auto-mpg.csv',header=None)
df.columns=['mpg','cylinders','displacement','horsepower','weigh','acceleration','model year','origin','name']

df['horsepower'].replace('?',np.nan, inplace=True)
df.dropna(subset=['horsepower'],axis=0,inplace=True)
df['horsepower']=df['horsepower'].astype('float')

#경계값 리스트 구하기
count, bin_dividers=np.histogram(df['horsepower'],bins=3)
print(bin_dividers)

[ 46.         107.33333333 168.66666667 230.        ]


In [47]:
bin_names=['저출력','보통출력','고출력']

df['hp_bin'] = pd.cut(x=df['horsepower'],
                     bins=bin_dividers,
                     labels=bin_names,
                     include_lowest=True)

print(df[['horsepower','hp_bin']].head(15))

    horsepower hp_bin
0        130.0   보통출력
1        165.0   보통출력
2        150.0   보통출력
3        150.0   보통출력
4        140.0   보통출력
5        198.0    고출력
6        220.0    고출력
7        215.0    고출력
8        225.0    고출력
9        190.0    고출력
10       170.0    고출력
11       160.0   보통출력
12       150.0   보통출력
13       225.0    고출력
14        95.0    저출력


(2) 더미변수

In [49]:
horsepower_dummies=pd.get_dummies(df['hp_bin'])
print(horsepower_dummies.head(15))

    저출력  보통출력  고출력
0     0     1    0
1     0     1    0
2     0     1    0
3     0     1    0
4     0     1    0
5     0     0    1
6     0     0    1
7     0     0    1
8     0     0    1
9     0     0    1
10    0     0    1
11    0     1    0
12    0     1    0
13    0     0    1
14    1     0    0


In [53]:
#원핫인코딩
from sklearn import preprocessing

label_encoder=preprocessing.LabelEncoder()
onehot_encoder=preprocessing.OneHotEncoder()

onehot_labeled=label_encoder.fit_transform(df['hp_bin'].head(15))  #숫자형 범주로 변환
print(onehot_labeled)
print(type(onehot_labeled))

#2차원 행렬로 형태 변형
onehot_reshaped=onehot_labeled.reshape(len(onehot_labeled),1)
print(onehot_reshaped)
print(type(onehot_reshaped))

onehot_fitted=onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)
print(type(onehot_fitted))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2]
<class 'numpy.ndarray'>
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]]
<class 'numpy.ndarray'>
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 1)	1.0
  (4, 1)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
<class 'scipy.sparse.csr.csr_matrix'>
